# Python Collections Module

The `collections` module provides specialized container datatypes that are alternatives to Python's built-in containers (dict, list, set, tuple).

**This notebook covers:**
1. `Counter` - Counting hashable objects
2. `defaultdict` - Dict with default factory
3. `deque` - Double-ended queue
4. `namedtuple` - Tuple with named fields
5. `OrderedDict` - Dict that remembers insertion order
6. `ChainMap` - Combining multiple mappings

All examples are focused on Data Engineering use cases.

In [ ]:
# Import all collections classes we'll use
from collections import Counter, defaultdict, deque, namedtuple, OrderedDict, ChainMap

---
# 1. Counter - Counting Objects

A `Counter` is a dict subclass for counting hashable objects.

**Syntax:**
```python
from collections import Counter

Counter(iterable)       # Count elements from iterable
Counter(mapping)        # Initialize from dict
Counter(keyword_args)   # Initialize from keyword args
```

**DE Use Cases:**
- Counting event occurrences in logs
- Frequency analysis of values
- Finding most common elements
- Data profiling (value distributions)

## 1.1 Basic Counter Usage

In [ ]:
# Creating a Counter from a list
events = ['click', 'view', 'click', 'purchase', 'view', 'click', 'view', 'view']

event_counts = Counter(events)

print(f"Events: {events}")
print(f"Counter: {event_counts}")
print(f"Type: {type(event_counts)}")

In [ ]:
# Counter behaves like a dict
print(f"Click count: {event_counts['click']}")
print(f"View count: {event_counts['view']}")

# Key difference: Missing keys return 0, not KeyError!
print(f"Unknown event count: {event_counts['unknown']}")

In [ ]:
# Counter from a string (counts characters)
text = "mississippi"
char_counts = Counter(text)

print(f"String: '{text}'")
print(f"Character counts: {char_counts}")

## 1.2 Counter Methods

In [ ]:
# most_common(n) - Returns n most frequent elements
log_levels = ['INFO', 'ERROR', 'INFO', 'WARNING', 'INFO', 'ERROR', 
              'INFO', 'INFO', 'DEBUG', 'ERROR', 'INFO', 'WARNING']

level_counts = Counter(log_levels)

print("Log level distribution:")
print(f"All counts: {level_counts}")
print(f"Top 2: {level_counts.most_common(2)}")
print(f"Top 3: {level_counts.most_common(3)}")

In [ ]:
# elements() - Returns iterator over elements (repeated by count)
c = Counter({'a': 3, 'b': 2, 'c': 1})

print(f"Counter: {c}")
print(f"Elements: {list(c.elements())}")

In [ ]:
# update() - Add counts from another iterable or counter
morning_events = Counter(['click', 'view', 'click'])
afternoon_events = Counter(['purchase', 'view', 'click', 'view'])

print(f"Morning: {morning_events}")
print(f"Afternoon: {afternoon_events}")

# Combine counts
morning_events.update(afternoon_events)
print(f"Combined: {morning_events}")

In [ ]:
# subtract() - Subtract counts
inventory = Counter({'apple': 10, 'banana': 5, 'orange': 8})
sold = Counter({'apple': 3, 'banana': 2})

print(f"Inventory: {inventory}")
print(f"Sold: {sold}")

inventory.subtract(sold)
print(f"Remaining: {inventory}")

## 1.3 Counter Arithmetic

In [ ]:
# Counter supports +, -, &, | operations
c1 = Counter({'a': 3, 'b': 2, 'c': 1})
c2 = Counter({'a': 1, 'b': 3, 'd': 2})

print(f"c1: {c1}")
print(f"c2: {c2}")
print()
print(f"c1 + c2 (add counts):      {c1 + c2}")
print(f"c1 - c2 (subtract, drop ≤0): {c1 - c2}")
print(f"c1 & c2 (min of each):     {c1 & c2}")
print(f"c1 | c2 (max of each):     {c1 | c2}")

## 1.4 DE Example: Log Analysis

In [ ]:
# Analyzing log data
logs = [
    {"timestamp": "2024-01-15 10:00:00", "level": "INFO", "service": "api"},
    {"timestamp": "2024-01-15 10:00:01", "level": "ERROR", "service": "db"},
    {"timestamp": "2024-01-15 10:00:02", "level": "INFO", "service": "api"},
    {"timestamp": "2024-01-15 10:00:03", "level": "WARNING", "service": "api"},
    {"timestamp": "2024-01-15 10:00:04", "level": "ERROR", "service": "api"},
    {"timestamp": "2024-01-15 10:00:05", "level": "INFO", "service": "db"},
    {"timestamp": "2024-01-15 10:00:06", "level": "ERROR", "service": "db"},
    {"timestamp": "2024-01-15 10:00:07", "level": "INFO", "service": "cache"},
]

# Count by log level
level_counts = Counter(log["level"] for log in logs)

# Count by service
service_counts = Counter(log["service"] for log in logs)

# Count errors by service
error_by_service = Counter(
    log["service"] for log in logs if log["level"] == "ERROR"
)

print("=" * 40)
print("LOG ANALYSIS REPORT")
print("=" * 40)
print(f"\nBy Level: {dict(level_counts)}")
print(f"By Service: {dict(service_counts)}")
print(f"Errors by Service: {dict(error_by_service)}")
print(f"\nMost problematic service: {error_by_service.most_common(1)[0][0]}")

---
# 2. defaultdict - Dict with Default Values

A `defaultdict` automatically creates missing keys with a default value.

**Syntax:**
```python
from collections import defaultdict

defaultdict(default_factory)
           ↓
           └── A callable that returns the default value
               Common: list, int, set, str, lambda: value
```

**DE Use Cases:**
- Grouping records by key
- Building inverted indexes
- Aggregating values
- Avoiding KeyError checks

## 2.1 Basic defaultdict Usage

In [ ]:
# Problem: Grouping items (regular dict approach)
orders = [
    {"customer": "C001", "product": "laptop"},
    {"customer": "C002", "product": "mouse"},
    {"customer": "C001", "product": "keyboard"},
    {"customer": "C003", "product": "monitor"},
    {"customer": "C001", "product": "mouse"},
]

# Regular dict - need to check if key exists
orders_by_customer_regular = {}
for order in orders:
    customer = order["customer"]
    if customer not in orders_by_customer_regular:
        orders_by_customer_regular[customer] = []
    orders_by_customer_regular[customer].append(order["product"])

print("Regular dict approach:")
for customer, products in orders_by_customer_regular.items():
    print(f"  {customer}: {products}")

In [ ]:
# defaultdict approach - cleaner!
orders_by_customer = defaultdict(list)

for order in orders:
    # No need to check if key exists - automatically creates empty list
    orders_by_customer[order["customer"]].append(order["product"])

print("defaultdict approach:")
for customer, products in orders_by_customer.items():
    print(f"  {customer}: {products}")

## 2.2 Different Default Factories

In [ ]:
# defaultdict(int) - Default value is 0
# Great for counting!

word_count = defaultdict(int)
words = ["apple", "banana", "apple", "cherry", "banana", "apple"]

for word in words:
    word_count[word] += 1  # No KeyError on first access

print(f"Word counts: {dict(word_count)}")

In [ ]:
# defaultdict(set) - Default value is empty set
# Great for collecting unique values!

transactions = [
    {"customer": "C001", "category": "electronics"},
    {"customer": "C001", "category": "clothing"},
    {"customer": "C001", "category": "electronics"},  # Duplicate
    {"customer": "C002", "category": "food"},
    {"customer": "C002", "category": "electronics"},
]

categories_by_customer = defaultdict(set)

for txn in transactions:
    categories_by_customer[txn["customer"]].add(txn["category"])

print("Unique categories per customer:")
for customer, categories in categories_by_customer.items():
    print(f"  {customer}: {categories}")

In [ ]:
# defaultdict with lambda - Custom default value

# Default value is a dict with initial structure
customer_stats = defaultdict(lambda: {"total_orders": 0, "total_amount": 0})

orders = [
    {"customer": "C001", "amount": 100},
    {"customer": "C002", "amount": 200},
    {"customer": "C001", "amount": 150},
    {"customer": "C001", "amount": 75},
]

for order in orders:
    customer_stats[order["customer"]]["total_orders"] += 1
    customer_stats[order["customer"]]["total_amount"] += order["amount"]

print("Customer statistics:")
for customer, stats in customer_stats.items():
    print(f"  {customer}: {stats}")

## 2.3 Nested defaultdict

In [ ]:
# Nested defaultdict for multi-level grouping
# Example: Group sales by region → product → sum amounts

sales = [
    {"region": "east", "product": "laptop", "amount": 1000},
    {"region": "west", "product": "laptop", "amount": 1200},
    {"region": "east", "product": "phone", "amount": 500},
    {"region": "east", "product": "laptop", "amount": 800},
    {"region": "west", "product": "phone", "amount": 600},
]

# Nested: region → product → total
sales_by_region_product = defaultdict(lambda: defaultdict(int))

for sale in sales:
    sales_by_region_product[sale["region"]][sale["product"]] += sale["amount"]

print("Sales by Region and Product:")
for region, products in sales_by_region_product.items():
    print(f"  {region}:")
    for product, total in products.items():
        print(f"    {product}: ${total}")

---
# 3. deque - Double-Ended Queue

A `deque` (pronounced "deck") is a double-ended queue with O(1) append/pop from both ends.

**Syntax:**
```python
from collections import deque

deque([iterable[, maxlen]])
                    ↓
                    └── Optional: maximum size (auto-discards old items)
```

**DE Use Cases:**
- Implementing queues (FIFO)
- Implementing stacks (LIFO)
- Sliding window calculations
- Recent items cache (with maxlen)
- Log buffering

## 3.1 Basic deque Operations

In [ ]:
# Creating a deque
d = deque([1, 2, 3])
print(f"Initial deque: {d}")

# Add to right (like list.append)
d.append(4)
print(f"After append(4): {d}")

# Add to left (O(1) - unlike list which is O(n))
d.appendleft(0)
print(f"After appendleft(0): {d}")

# Remove from right
right = d.pop()
print(f"After pop(): {d} (removed: {right})")

# Remove from left (O(1) - unlike list which is O(n))
left = d.popleft()
print(f"After popleft(): {d} (removed: {left})")

In [ ]:
# List vs Deque performance for left operations
import time

# List: append to left is O(n)
lst = []
start = time.time()
for i in range(100000):
    lst.insert(0, i)  # O(n) each time!
list_time = time.time() - start

# Deque: append to left is O(1)
dq = deque()
start = time.time()
for i in range(100000):
    dq.appendleft(i)  # O(1) each time!
deque_time = time.time() - start

print(f"List insert(0, x) 100k times: {list_time:.3f}s")
print(f"Deque appendleft(x) 100k times: {deque_time:.3f}s")
print(f"Deque is {list_time/deque_time:.0f}x faster!")

## 3.2 deque with maxlen - Rolling Window / Buffer

In [ ]:
# maxlen creates a fixed-size buffer
# When full, adding new items automatically discards old ones

recent_events = deque(maxlen=5)  # Keep only last 5 events

events = ['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8']

print("Adding events to maxlen=5 deque:")
for event in events:
    recent_events.append(event)
    print(f"  After adding {event}: {list(recent_events)}")

In [ ]:
# DE Use Case: Rolling average of last N values

def rolling_average(data, window_size):
    """Calculate rolling average using deque."""
    window = deque(maxlen=window_size)
    averages = []
    
    for value in data:
        window.append(value)
        avg = sum(window) / len(window)
        averages.append(round(avg, 2))
    
    return averages

# Test with stock prices
prices = [100, 102, 104, 103, 105, 107, 106, 108, 110, 109]
rolling_avg = rolling_average(prices, window_size=3)

print("Stock Price Analysis (3-day rolling average):")
print("-" * 40)
for i, (price, avg) in enumerate(zip(prices, rolling_avg)):
    print(f"Day {i+1}: Price=${price}, Rolling Avg=${avg}")

## 3.3 deque as Queue (FIFO) and Stack (LIFO)

In [ ]:
# Queue (FIFO): First In, First Out
# Add to right, remove from left

job_queue = deque()

# Add jobs
job_queue.append("job_1")
job_queue.append("job_2")
job_queue.append("job_3")

print("Queue (FIFO):")
print(f"Jobs: {list(job_queue)}")

# Process jobs in order
while job_queue:
    job = job_queue.popleft()  # Remove from left
    print(f"Processing: {job}")

In [ ]:
# Stack (LIFO): Last In, First Out
# Add to right, remove from right

undo_stack = deque()

# Add actions
undo_stack.append("action_1")
undo_stack.append("action_2")
undo_stack.append("action_3")

print("Stack (LIFO):")
print(f"Actions: {list(undo_stack)}")

# Undo in reverse order
while undo_stack:
    action = undo_stack.pop()  # Remove from right
    print(f"Undoing: {action}")

## 3.4 Other deque Methods

In [ ]:
# rotate(n) - Rotate elements
d = deque([1, 2, 3, 4, 5])
print(f"Original: {d}")

d.rotate(2)  # Rotate right by 2
print(f"Rotate right by 2: {d}")

d.rotate(-2)  # Rotate left by 2 (back to original)
print(f"Rotate left by 2: {d}")

In [ ]:
# extend() and extendleft()
d = deque([3, 4, 5])
print(f"Original: {d}")

d.extend([6, 7])  # Add to right
print(f"After extend([6, 7]): {d}")

d.extendleft([2, 1])  # Add to left (note: reversed!)
print(f"After extendleft([2, 1]): {d}")
print("Note: extendleft adds items one by one, so order is reversed")

---
# 4. namedtuple - Tuple with Named Fields

A `namedtuple` creates tuple subclasses with named fields.

**Syntax:**
```python
from collections import namedtuple

TypeName = namedtuple('TypeName', ['field1', 'field2', ...])
# OR
TypeName = namedtuple('TypeName', 'field1 field2 ...')
```

**DE Use Cases:**
- Creating lightweight record types
- Returning multiple values with names
- Representing database rows
- Configuration objects

## 4.1 Basic namedtuple Usage

In [ ]:
# Define a namedtuple for database records
Customer = namedtuple('Customer', ['id', 'name', 'email', 'city'])

# Create instances
c1 = Customer('C001', 'John Doe', 'john@example.com', 'New York')
c2 = Customer(id='C002', name='Jane Smith', email='jane@example.com', city='Los Angeles')

print(f"Customer 1: {c1}")
print(f"Customer 2: {c2}")

In [ ]:
# Access by name (more readable than tuple indices)
print(f"Name: {c1.name}")
print(f"Email: {c1.email}")
print(f"City: {c1.city}")

# Still works with tuple indexing
print(f"\nBy index: {c1[0]}, {c1[1]}")

In [ ]:
# namedtuple is immutable (like regular tuple)
try:
    c1.name = "New Name"
except AttributeError as e:
    print(f"Cannot modify: {e}")

## 4.2 namedtuple Methods

In [ ]:
# _make() - Create from iterable
csv_row = ['C003', 'Bob Johnson', 'bob@example.com', 'Chicago']
c3 = Customer._make(csv_row)
print(f"From CSV: {c3}")

In [ ]:
# _asdict() - Convert to dictionary
c1_dict = c1._asdict()
print(f"As dict: {c1_dict}")
print(f"Type: {type(c1_dict)}")

In [ ]:
# _replace() - Create new instance with some fields changed
c1_updated = c1._replace(city='Boston', email='john.new@example.com')

print(f"Original: {c1}")
print(f"Updated:  {c1_updated}")

In [ ]:
# _fields - Get field names
print(f"Fields: {Customer._fields}")

## 4.3 DE Example: Processing Database Records

In [ ]:
# Define record type for transactions
Transaction = namedtuple('Transaction', ['txn_id', 'customer_id', 'amount', 'status', 'date'])

# Simulated database rows
raw_data = [
    ('T001', 'C001', 150.00, 'completed', '2024-01-15'),
    ('T002', 'C002', 200.00, 'pending', '2024-01-15'),
    ('T003', 'C001', 75.50, 'completed', '2024-01-16'),
    ('T004', 'C003', 300.00, 'failed', '2024-01-16'),
    ('T005', 'C002', 125.00, 'completed', '2024-01-17'),
]

# Convert to namedtuples
transactions = [Transaction._make(row) for row in raw_data]

# Now we can access fields by name!
print("Completed transactions:")
for txn in transactions:
    if txn.status == 'completed':
        print(f"  {txn.txn_id}: Customer {txn.customer_id}, Amount ${txn.amount}")

# Calculate total by status
completed_total = sum(t.amount for t in transactions if t.status == 'completed')
print(f"\nTotal completed: ${completed_total}")

---
# 5. OrderedDict - Ordered Dictionary

**Note:** Since Python 3.7+, regular `dict` maintains insertion order. However, `OrderedDict` still has some unique features.

**Unique OrderedDict features:**
- `move_to_end()` method
- Equality comparison considers order
- `popitem(last=True/False)` to pop from either end

In [ ]:
# OrderedDict maintains insertion order (like regular dict in Python 3.7+)
od = OrderedDict()
od['first'] = 1
od['second'] = 2
od['third'] = 3

print(f"OrderedDict: {od}")

In [ ]:
# move_to_end() - Unique to OrderedDict
od = OrderedDict([('a', 1), ('b', 2), ('c', 3), ('d', 4)])
print(f"Original: {od}")

od.move_to_end('b')  # Move to end
print(f"After move_to_end('b'): {od}")

od.move_to_end('d', last=False)  # Move to beginning
print(f"After move_to_end('d', last=False): {od}")

In [ ]:
# popitem() - Pop from either end
od = OrderedDict([('a', 1), ('b', 2), ('c', 3)])
print(f"Original: {od}")

last = od.popitem(last=True)  # Pop from end (default)
print(f"Popped last: {last}, Remaining: {od}")

first = od.popitem(last=False)  # Pop from beginning
print(f"Popped first: {first}, Remaining: {od}")

In [ ]:
# Equality considers order in OrderedDict, not in regular dict
od1 = OrderedDict([('a', 1), ('b', 2)])
od2 = OrderedDict([('b', 2), ('a', 1)])

d1 = {'a': 1, 'b': 2}
d2 = {'b': 2, 'a': 1}

print(f"OrderedDict comparison (order matters): {od1 == od2}")
print(f"Regular dict comparison (order ignored): {d1 == d2}")

---
# 6. ChainMap - Combining Multiple Dicts

A `ChainMap` groups multiple dicts together for lookup.

**Syntax:**
```python
from collections import ChainMap

ChainMap(dict1, dict2, dict3, ...)
```

**Key behavior:** Lookups search dicts in order, returns first match.

**DE Use Cases:**
- Configuration with defaults and overrides
- Layered settings (user → project → global)
- Combining multiple data sources

In [ ]:
# ChainMap for layered configuration

# Default settings
defaults = {
    'host': 'localhost',
    'port': 5432,
    'database': 'default_db',
    'timeout': 30
}

# User overrides (only override some settings)
user_config = {
    'host': 'production.server.com',
    'database': 'prod_db'
}

# Environment overrides (highest priority)
env_config = {
    'timeout': 60
}

# Combine: env_config > user_config > defaults
config = ChainMap(env_config, user_config, defaults)

print("Effective configuration:")
for key in ['host', 'port', 'database', 'timeout']:
    print(f"  {key}: {config[key]}")

In [ ]:
# ChainMap.maps - Access underlying dicts
print(f"Number of maps: {len(config.maps)}")
print(f"Maps: {config.maps}")

In [ ]:
# new_child() - Add a new layer at the front
runtime_override = {'database': 'test_db'}  # For testing

test_config = config.new_child(runtime_override)

print(f"Original database: {config['database']}")
print(f"Test database: {test_config['database']}")

---
# Quick Reference: Collections Module

| Class | Description | DE Use Case |
|-------|-------------|-------------|
| `Counter` | Count occurrences | Log analysis, frequency distribution |
| `defaultdict` | Dict with default factory | Grouping, aggregation without KeyError |
| `deque` | Double-ended queue | Queues, stacks, sliding windows |
| `namedtuple` | Tuple with named fields | Database records, config objects |
| `OrderedDict` | Dict with order methods | When order-aware operations needed |
| `ChainMap` | Combine multiple dicts | Layered configuration |